In [0]:
# ======================================================================================
# BIBLIOTHÈQUES
# ======================================================================================
import logging
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, monotonically_increasing_id

# ======================================================================================
# 1. DÉCLARATION DES PARAMÈTRES (WIDGETS)
# ======================================================================================
dbutils.widgets.text("storage_account", "stsalesinsightcuxm0611", "Nom du compte de stockage")
dbutils.widgets.text("container", "data", "Nom du conteneur")
dbutils.widgets.text("silver_folder", "silver/sales_orders/", "Dossier source dans la couche Silver")

# --- Modification : Utilisation d'un scope de secrets unique pour tout le projet ---
dbutils.widgets.text("secret_scope", "dbricks-scope-projet", "Scope unique pour les secrets du projet")

# Paramètres pour les secrets
dbutils.widgets.text("adls_secret_key", "adls-access-key", "Clé du secret pour l'accès ADLS")
dbutils.widgets.text("sql_user_key", "sql-admin-user", "Clé du secret pour l'utilisateur SQL")
dbutils.widgets.text("sql_password_key", "sql-admin-password", "Clé du secret pour le mot de passe SQL")

# Paramètres pour la connexion à Azure SQL
dbutils.widgets.text("jdbc_hostname", "sqlsvr-salesinsightcuxm0611.database.windows.net", "Serveur Azure SQL DB")
dbutils.widgets.text("jdbc_database", "sqldb-salesinsight-gold", "Base de données Gold")


# ======================================================================================
# 2. DÉFINITION DES FONCTIONS
# ======================================================================================

def setup_adls_access(storage_account: str, scope: str, key_name: str) -> None:
    """
    Configure l'accès au compte de stockage ADLS Gen2 pour la session Spark en cours.
    C'est la méthode de connexion directe, utilisée quand le montage n'est pas possible.
    :param storage_account: Nom du compte de stockage.
    :param scope: Nom du Secret Scope où la clé est stockée.
    :param key_name: Nom de la clé dans le Secret Scope.
    """
    logging.info(f"Configuration de l'accès pour le compte de stockage : {storage_account}")
    access_key = dbutils.secrets.get(scope=scope, key=key_name)
    spark.conf.set(
        f"fs.azure.account.key.{storage_account}.dfs.core.windows.net",
        access_key
    )
    logging.info("Accès ADLS configuré avec succès pour cette session.")

def read_silver_data(source_path: str) -> DataFrame:
    """
    Lit la table Delta depuis la couche Silver.
    :param source_path: Chemin complet vers les données sources.
    :return: DataFrame Spark contenant les données Silver.
    """
    logging.info(f"Lecture des données Silver depuis : {source_path}")
    try:
        df = spark.read.format("delta").load(source_path)
        logging.info(f"Lecture réussie depuis la couche Silver. {df.count()} lignes chargées.")
        return df
    except Exception as e:
        logging.error(f"❌ ERREUR lors de la lecture des données depuis {source_path}", exc_info=True)
        raise e

def transform_to_dim_product(silver_df: DataFrame) -> DataFrame:
    """
    Crée la dimension Produit à partir des données Silver.
    :param silver_df: DataFrame contenant les données Silver.
    :return: DataFrame transformé pour la DimProduct.
    """
    logging.info("Début de la création de la dimension 'DimProduct'.")
    df_dim_product = silver_df.select("PRODUCTCODE", "PRODUCTLINE", "MSRP").distinct()
    df_dim_product = df_dim_product.withColumn("ProductKey", monotonically_increasing_id())
    df_dim_product_final = df_dim_product.select("ProductKey", "PRODUCTCODE", "PRODUCTLINE", "MSRP")
    logging.info("Transformation vers 'DimProduct' terminée.")
    return df_dim_product_final

def get_jdbc_connection_properties(hostname: str, db_name: str, scope: str, user_key: str, pwd_key: str) -> tuple[str, dict]:
    """
    Construit l'URL JDBC et le dictionnaire de propriétés pour la connexion à Azure SQL DB.
    :return: Un tuple contenant (jdbc_url, connection_properties).
    """
    logging.info("Configuration de la connexion JDBC.")
    jdbc_port = 1433
    sql_user = dbutils.secrets.get(scope=scope, key=user_key)
    sql_password = dbutils.secrets.get(scope=scope, key=pwd_key)
    jdbc_url = f"jdbc:sqlserver://{hostname}:{jdbc_port};database={db_name}"
    properties = {
      "user": sql_user,
      "password": sql_password,
      "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
    }
    return jdbc_url, properties

def write_dimension_to_gold(dim_df: DataFrame, table_name: str, jdbc_url: str, properties: dict) -> None:
    """
    Écrit un DataFrame de dimension dans la couche Gold (Azure SQL DB).
    :param dim_df: DataFrame de la dimension à écrire.
    :param table_name: Nom de la table de destination.
    :param jdbc_url: URL de connexion JDBC.
    :param properties: Propriétés de connexion.
    """
    logging.info(f"Début de l'écriture de la dimension '{table_name}' vers la couche Gold.")
    try:
        dim_df.write.jdbc(
            url=jdbc_url,
            table=table_name,
            mode="overwrite",
            properties=properties
        )
        logging.info(f"✅ La dimension '{table_name}' a été écrite avec succès dans la couche Gold.")
    except Exception as e:
        logging.error(f"❌ ERREUR lors de l'écriture de la dimension '{table_name}'.", exc_info=True)
        raise e

# ======================================================================================
# 3. POINT D'ENTRÉE PRINCIPAL (MAIN)
# Orchestration des appels de fonctions.
# ======================================================================================
if __name__ == "__main__":
    
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    
    logging.info("===================================================")
    logging.info("DÉMARRAGE DU PIPELINE SILVER-TO-GOLD (DimProduct)")
    logging.info("===================================================")
    
    try:
        # Récupération des paramètres
        storage_account = dbutils.widgets.get("storage_account").strip()
        container = dbutils.widgets.get("container").strip()
        silver_folder = dbutils.widgets.get("silver_folder").strip()
        secret_scope = dbutils.widgets.get("secret_scope").strip()
        adls_secret_key = dbutils.widgets.get("adls_secret_key").strip()
        jdbc_hostname = dbutils.widgets.get("jdbc_hostname").strip()
        jdbc_database = dbutils.widgets.get("jdbc_database").strip()
        sql_user_key = dbutils.widgets.get("sql_user_key").strip()
        sql_password_key = dbutils.widgets.get("sql_password_key").strip()

        # --- ORCHESTRATION ---

        # 1. Configurer l'accès au Data Lake
        setup_adls_access(storage_account, secret_scope, adls_secret_key)

        # 2. Configurer la connexion à la base SQL Gold
        source_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/{silver_folder}"
        jdbc_url, connection_props = get_jdbc_connection_properties(
            jdbc_hostname, jdbc_database, secret_scope, sql_user_key, sql_password_key
        )
        
        # 3. Exécuter le pipeline ETL
        silver_dataframe = read_silver_data(source_path)
        dim_product_dataframe = transform_to_dim_product(silver_dataframe)
        write_dimension_to_gold(dim_product_dataframe, "DimProduct", jdbc_url, connection_props)
        
        logging.info("===================================================")
        logging.info("PIPELINE SILVER-TO-GOLD (DimProduct) TERMINÉ AVEC SUCCÈS")
        logging.info("===================================================")

    except Exception as e:
        logging.error("Le pipeline a échoué dans le bloc principal.", exc_info=True)
        raise e
